In [1]:
import uuid

import babel

from seis_lab_data import (
    config,
    operations,
    schemas,
)
from seis_lab_data.db.engine import (
    get_engine,
    get_session_maker,
)
from seis_lab_data.db import queries

EN_LOCALE = babel.Locale.parse("en_US")
PT_LOCALE = babel.Locale.parse("pt_PT")

settings = config.get_settings()

engine = get_engine(settings.database_dsn.unicode_string(), settings.debug)
session_maker = get_session_maker(engine)

/home/ubuntu/app/.venv/lib/python3.12/site-packages/pydantic_settings/sources/providers/secrets.py:60: UserWarning: directory "/run/secrets" does not exist
  warnings.warn(f'directory "{path}" does not exist')


In [4]:
async with session_maker() as session:
    c1 = await queries.get_project_by_english_name(session, "proj1")
    print(c1.model_dump())

2025-09-19 10:02:52,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 10:02:52,349 INFO sqlalchemy.engine.Engine SELECT project.id, project.owner, project.name, project.description, project.status, project.root_path, project.is_valid, project.links 
FROM project 
WHERE (project.name ->> %(name_1)s::TEXT) = %(param_1)s::VARCHAR
2025-09-19 10:02:52,349 INFO sqlalchemy.engine.Engine [cached since 21.12s ago] {'name_1': 'en', 'param_1': 'proj1'}
{'name': {'en': 'proj1', 'pt': ''}, 'is_valid': False, 'links': [], 'owner': '7bc0548d9be29526d15357d6c13efae7dc86c145a10e04e30bcad7d7926349fd', 'id': UUID('a7773b1b-7611-468b-af47-098d9361884d'), 'description': {'en': '', 'pt': ''}, 'status': <ProjectStatus.DRAFT: 'draft'>, 'root_path': ''}
2025-09-19 10:02:52,351 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
c1.model_dump()

{'name': {'en': 'My first campaign', 'pt': 'A minha primeira campanha'},
 'slug': 'my-first-campaign',
 'root_path': '/projects/first',
 'links': [{'url': 'https://fakeurl.com',
   'relation': 'related',
   'media_type': 'text/html',
   'description': {'en': 'A fake description for link',
    'pt': 'Uma descrição falsa para o link'}}],
 'description': {'en': 'A fake description for my first campaign',
  'pt': 'Uma descrição sintética para a minha primeira campanha'},
 'id': UUID('74f07051-1aa9-4c08-bc27-3ecf101ab5b3'),
 'owner': 'fakeowner1',
 'status': <MarineCampaignStatus.DRAFT: 'draft'>,
 'is_valid': False}

In [4]:
schemas.ProjectReadDetail(**c1.model_dump())

MarineCampaignReadDetail(id=UUID('74f07051-1aa9-4c08-bc27-3ecf101ab5b3'), slug='my-first-campaign', name={'en': 'My first campaign', 'pt': 'A minha primeira campanha'}, description={'en': 'A fake description for my first campaign', 'pt': 'Uma descrição sintética para a minha primeira campanha'}, status=<MarineCampaignStatus.DRAFT: 'draft'>, is_valid=False, owner='fakeowner1', root_path='/projects/first', links=[LinkSchema(url='https://fakeurl.com', media_type='text/html', relation='related', description={'en': 'A fake description for link', 'pt': 'Uma descrição falsa para o link'})])

In [2]:
to_create = schemas.ProjectCreate(
    id=uuid.uuid4(),
    owner="some-guy",
    name={
        EN_LOCALE.language: "my beautiful campaign",
        PT_LOCALE.language: "A minha belíssima campanha",
    },
    links=[
        {
            "url": "http://fake.com",
            "media_type": "text/html",
            "relation": "parent",
            "description": {EN_LOCALE.language: "This is a link about something"},
        }
    ],
)

In [3]:
async with session_maker() as session:
    c1 = await operations.create_project(to_create, settings)

2025-08-27 16:49:10,594 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-27 16:49:10,594 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,596 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-27 16:49:10,596 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,597 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-27 16:49:10,598 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,602 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-27 16:49:10,603 INFO sqlalchemy.engine.Engine INSERT INTO marinecampaign (id, owner, name, slug, links) VALUES (%(id)s::UUID, %(owner)s::VARCHAR, %(name)s::JSONB, %(slug)s::VARCHAR, %(links)s::JSONB)
2025-08-27 16:49:10,604 INFO sqlalchemy.engine.Engine [generated in 0.00053s] {'id': UUID('232c2349-6adb-4b79-bcea-88732b514d07'), 'owner': 'some-guy', 'name': Jsonb({'en': 'my beautiful campaign', 'pt ... (67 chars)), 'slug': 'my-beautiful-campaign', 'links': Jsonb([{'url